# Sheet 02

## Preamble

Autors: Marten Ringwelski, Nico Ostermann, Simon Liessem

Note that this notebook MUST be executed in order to get everything to work.
The tasks can't be run individually. 

Also eCampus does not allow for uploading nested directory structures which makes it hard to properly organize the files. The files are expected to be in the `data` directory which itself is placed next to this notebook.

If you extract the zip file we handed in everything should work just fine.

Autoformatting if `jupyter-black` is installed.

In [ ]:
try:
    import black
    import jupyter_black

    jupyter_black.load(
        lab=False,
        line_length=79,
        verbosity="DEBUG",
        target_version=black.TargetVersion.PY310,
    )
except ImportError:
    pass

Import all we weed and more.

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.feature_selection import f_classif, SelectKBest
import math as m

Set seaborn default theme

In [ ]:
sns.set_theme()

If needed tweak parameters of matplotlib.
Here we increase the size and dpi to bet a bigger but still high-res image.

In [ ]:
mpl.rcParams["figure.dpi"] = 200
mpl.rcParams["figure.figsize"] = (20, 15)
%matplotlib inline

## Exercise 1

### a)

First create the figure and axes.

XXX Don't display the plot just jet.

In [ ]:
fig, ax = plt.subplots(
    nrows=1,
    ncols=1,
)

In [ ]:
df = pd.read_csv(
    "data/values.txt",
    names=["data"],
)

Based on the following kde we assume that the maximum is at about 1.5.

In [ ]:
sns.kdeplot(df)

### b)

In [ ]:
step = 0.25
colors = ["orange", "green", "blue", "yellow"]
for adjust, color in zip(np.arange(step, 1 + step, step), colors):
    sns.kdeplot(
        df,
        bw_adjust=adjust,
        ax=ax,
        label=f"Adjust = {adjust}",
        # XXX Why doesn't this work?
        c=color,
    )

In [ ]:
sns.histplot(
    df,
    binwidth=0.3,
    stat="density",
    ax=ax,
    label="Histogram",
)

In [ ]:
ax.legend()

Show the KDE's and a histogram.
They all have the same color, which is not intendet.
This seems to be a bug in seaborn as specifying a `drawstyle` does not work either.

Still despite the colors you can easily distinguish the graphs.
We see that with a smaller scale (smaller stdandart derivation) the amount of modes increases to 4 with a scale of 0.25.

The histogram shows the same four modes so we suggest that the underlying distribution has 4 modes which are shown in the picture ($x \in \{ -0.25, 1, 1.75, 3\}$)

In [ ]:
fig

## Exercise 2

### a)

In [ ]:
df = pd.read_excel(
    "data/chronic_kidney_disease_numerical.xls",
)
columns = df.columns.tolist()
columns.remove("class")
df = df.melt(id_vars=["class"])

### b)

In [ ]:
fig, axs = plt.subplots(
    nrows=2,
    ncols=m.ceil(len(columns) / 2),
)

for column, ax in zip(columns, axs.flatten()):
    sns.boxplot(
        df,
        x=df["class"],
        y=df[df["variable"] == column]["value"],
        ax=ax,
    )
    ax.set_xlabel(column)

## Exercise 3

### a)

In [ ]:
df = pd.read_csv(
    "data/winequality-red.csv",
    sep=";",
)

In [ ]:
df.head()

### b)

In [ ]:
fig, ax = plt.subplots(
    nrows=1,
    ncols=1,
)

In [ ]:
df.hist(
    column="quality",
    ax=ax,
)

From the following plot we see that the minimum quality is 3 and the maximum is 8.

In [ ]:
fig

### c)

In [ ]:
min_quality = df["quality"].min()
max_quality = df["quality"].max()
# 'Replace the original “quality” column with a new column “quality bin”'
# The task is unclrear, we descite to actually replace.
df["quality"] = pd.cut(
    df["quality"],
    bins=[min_quality, min_quality + 1, max_quality - 2, max_quality],
    labels=["low", "medium", "high"],
    include_lowest=True,
    right=True,
)

### d)

In [ ]:
# We need a copy here as we modify the category
df_wo_medium = df[df["quality"] != "medium"].copy()
df_wo_medium["quality"] = df_wo_medium[
    "quality"
].cat.remove_unused_categories()

### e)

The produced plot is really large and for my my browser struggles to display it.
A workaround is saving the image to disk and viewing it with a proper image viewer (e.g. imv).

In [ ]:
sns.pairplot(
    df_wo_medium,
    hue="quality",
    diag_kind="kde",
)

### f)

Our approach is do notice differences in the distribution of variables.
So critic acid, alcohol, sulphates are obvious choices.
For the last two we chose residual sugar and pH because the distributions also differ, but not that much.

### g)

In [ ]:
fit = SelectKBest(
    score_func=f_classif,
    k=5,
).fit(
    X=df_wo_medium.drop(columns="quality"),
    # Wtf sklearn, why can't you take categorical data?!
    y=df_wo_medium["quality"].astype(str),
)
features = fit.get_feature_names_out().tolist()

The top five attributes according to F-Score are the same as we chose.

In [ ]:
features

In [ ]:
df_kbest = df_wo_medium[features + ["quality"]].copy()

### h)

In [ ]:
pair_grid = sns.PairGrid(
    df_kbest,
    vars=features,
    hue="quality",
)
pair_grid.map_diag(sns.kdeplot)
pair_grid.map_lower(sns.regplot, scatter=False)
pair_grid.map_upper(sns.scatterplot)
pair_grid.add_legend()

### i)

In the figure we can see that sulphates-alcohol and ph-citric acid seem to correlated regardless of quality. Also pH-volatile acidity and citric acid-volatile acidity appear to correlate, but in both cases the values from higher quality wines seem to correlate a bit more. 
For sulphates-citric acid and sulphates-pH the correlation is dependend on the quality of the wines. In both cases only low quality wines seem to have a high correlation, while the high quality wines have a low correlation.
The attribute that seems to have a multimodal distribution is the citric acid. It seems to have a peak at 0 and a peak at 0.5.

Now let us calculate the outliers programatically.
We consider a datapoint an outlier if it is outside 4 times the inter quantile range in any of the given attributes.
Suprisingly we get a lot of outliers.

In [ ]:
iqr = df_kbest.quantile(
    q=0.75,
    numeric_only=True,
) - df_kbest.quantile(
    q=0.25,
    numeric_only=True,
)
median = df_kbest.median(numeric_only=True)
lower_bound = median - 2.0 * iqr
upper_bound = median + 2.0 * iqr
df_kbest_wo_quality = df_kbest.drop(columns="quality")

is_outlier = np.logical_or(
    df_kbest_wo_quality.le(lower_bound).any(axis=1),
    df_kbest_wo_quality.ge(upper_bound).any(axis=1),
)

Now lets have a look the the outliers.

In [ ]:
df_kbest[is_outlier]

### j)

In [ ]:
def distance_consistency(data, by: str, subset: str | list[str] = None):
    """
    Calculate the distance consistency for existing clusters.

    Parameters
    ----------
    data : DataFrame
        The data to use.
    by : str
        Name of the column whose values define the clusters.
    subset: column label or list of column labels, optional
        Labels that are considered for the calculation.
    """
    if subset is None:
        pass
    elif isinstance(subset, str):
        data = data[[subset, by]]
    elif isinstance(subset, list):
        data = data[subset + [by]]
    else:
        raise ValueError(
            f"subset must be an instance of string of list but is {type(subset)}"
        )

    groupby_df = data.groupby(by, group_keys=False)
    # The index of this df are the groups
    centroids = groupby_df.mean()

    # This is kind of fancy and hopefully the simplest solution in terms of cognitive overhead and perfomance
    # Frist use apply to substract the respective cluster centroid
    # Then use apply again to calculate the norm
    # As indices are preserved the assigment works fine
    # We have to use another dataframe to store the distances as we would otherwise (obiviously) modify
    # groupby_df.
    distance_df = pd.DataFrame(index=data.index)
    distance_df["distance"] = groupby_df.apply(
        lambda group: group.drop(columns=by) - centroids.loc[group.name],
    ).apply(
        np.linalg.norm,
        axis=1,
    )

    distance_df["min_distance"] = distance_df["distance"]

    # Calculate the nearest centroid distance
    for other_group in groupby_df.groups:
        other_distance = groupby_df.apply(
            lambda group: group.drop(columns=by) - centroids.loc[other_group],
        ).apply(
            np.linalg.norm,
            axis=1,
        )
        distance_df["min_distance"] = pd.concat(
            [distance_df["distance"], other_distance],
            axis=1,
        ).min(axis=1)

    # Count how many points are closest to their own centroid
    other_centroid_closer = (
        distance_df["min_distance"] < distance_df["distance"]
    )
    own_centroid_closer_count = other_centroid_closer.value_counts().loc[False]
    n_rows = data.shape[0]

    return own_centroid_closer_count / n_rows

In [ ]:
import itertools as it

In [ ]:
distance_consistencies = {}

for labels in it.combinations(features, 2):
    labels = list(labels)
    distance_consistencies[" - ".join(labels)] = distance_consistency(
        df_kbest,
        by="quality",
        subset=labels,
    )

In [ ]:
distance_consistencies

In [ ]:
max(distance_consistencies, key=distance_consistencies.get)

From the above we see that the scatterplot of pH against sulphates has the highest distance consistency of roughly 96.42%.

## Exercise 4

Obviously a) and b) cite the paper.

### a)

The eight variations are the following:
1. Embedding scatterplots (SP)

Between every pair of variables in the PCP a scatterplot is drawn.
The plot is rotated by 45° to share axis.
They expect this to improve the visualisation because of the intuitivness of scatterplots.

2. Highlighting fuzzy clusters [using color] (Color)

They calculate a value ("enhancement") $e_i$ for each datapoint $i$.
This value correlates to fuzzy clusters and is used for coloring the PCP lines.
Color is considered as a strong visual indicator so the autors expect this to improve the visualisation.

3. Highlighting fuzzy clusters [using opacity] (Blend)

Works exactly like the second variation but the enhancement is used for determining the opacity rather than the colors of the lines.

4. Highlighting fuzzy clusters [using color & opacity] (ColorBlend)

This method combines the second and the forth method. The enhancement is used for setting the color and opacity.
This is expected to improve the the visualisations for the same reasons as the methods Blend and ColorBlend.

5. Curves (Curved)

This approach is a variation of a common approach called Polylines.
In contrast to Polylines in their approach resolves the problem of crossing ambiguity.
This should improve the visualistation as a single datapoint can be followed.

6.  Random Tour (RT)

They animate by generating a random permutation of the data axis and then swapping two axis at a time.
When the animation is done the axis are as generated in the random permutation.

7. Random Permutation (RP)

The idea is the same as in the 6. variation.
But the animataion swaps two axis so long that all possible permutations are show trouhout the animation.

8. Adding sinus (wobble)

This is also an animation based visualisation.
The polylines of points that are close together are wobbled by adding a sinus curve (depending on time of the animation)

### b)

The autors expected the vairations to perform in the following order:
ColorBlend, Color, SP, Curved, Blend, Wobble, {RT, PT}, Standard

Curly braces indicate that the variations in them did not show significant difference in performance.

The results did not match the hypothisis.
They interpret their results so that the performance is ordered as folows:
SP, {Standard, Color, Blend, ColorBlend, Curved, Wobble}, {RT, PT}

### c)

We would use the variation that addidionally draws scatterplots.
We base our choice on the result of their studies which showed that this variation was the only one significatly improving the response time of identifying clusters.